# Radical Pilot Tutorial


## 1. Installation (Not needed on Chameleon)

Not required on <http://sc15.radical-cybertools.org/>

If you want to install Radical-Pilot locally:

    pip install radical.pilot
    pip install radical.utils
    
## 2. Radical Pilot Setup

Import dependency and create callbacks for state update

Documentation: http://radicalpilot.readthedocs.org/en/latest/machconf.html#preconfigured-resources

In [6]:
import os, sys
import commands
import radical.pilot as rp
import pandas as pd
#os.environ["RADICAL_PILOT_DBURL"]="mongodb://ec2-54-221-194-147.compute-1.amazonaws.com:24242/giannis"
os.environ["RADICAL_PILOT_DBURL"]="mongodb://localhost:27017/radicalpilot"
os.environ["RADICAL_PILOT_VERBOSE"]="DEBUG"

In [2]:
commands.getstatusoutput('echo $RADICAL_PILOT_VERBOSE')

(0, 'DEBUG')

## 3. Submit Pilot

### 3.1 Create a new Session and Pilot-Manager. 

In [8]:
session = rp.Session()
c = rp.Context('ssh')
c.user_id = "radical"
session.add_context(c)
pmgr = rp.PilotManager(session=session)
umgr = rp.UnitManager (session=session,
                       scheduler=rp.SCHED_DIRECT_SUBMISSION)
print "Session id: %s Pilot Manager: %s" % (session.uid, str(pmgr.as_dict()))

Session id: rp.session.LMUCX29608.local.luckow.016741.0005 Pilot Manager: {'uid': 'pmgr.0001'}


### 3.2 Submit Pilot and initialize Unit Manager

In [11]:
pd.DataFrame(umgr.as_dict())

ValueError: If using all scalar values, you must pass an index

In [5]:
pdesc = rp.ComputePilotDescription ()
pdesc.resource = "'local.locahost_anaconda"  # NOTE: This is a "label", not a hostname
pdesc.runtime  = 10 # minutes
pdesc.cores    = 16
pdesc.cleanup  = False
pilot = pmgr.submit_pilots(pdesc)
umgr.add_pilots(pilot)

PilotException: Resource key ''local.locahost_anaconda' is not known.

### 3.3 Submit Compute Units

In [6]:
NUMBER_JOBS  = 2 # the total number of cus to run
cudesc_list = []
for i in range(NUMBER_JOBS):
    # -------- BEGIN USER DEFINED CU DESCRIPTION --------- #
    cudesc = rp.ComputeUnitDescription()
    cudesc.environment = {'CU_NO': i}
    cudesc.executable  = "/bin/echo"
    cudesc.arguments   = ['I am CU number $CU_NO']
    cudesc.cores       = 1
    # -------- END USER DEFINED CU DESCRIPTION --------- #
    cudesc_list.append(cudesc)

Submit the previously created ComputeUnit descriptions to the PilotManager. This will trigger the selected scheduler to start assigning ComputeUnits to the ComputePilots.

In [ ]:
# Submit the previously created ComputeUnit descriptions to the
# PilotManager. This will trigger the selected scheduler to start
# assigning ComputeUnits to the ComputePilots.
print "Submit Compute Units to Unit Manager ..."
cu_set = umgr.submit_units (cudesc_list)
print "Waiting for CUs to complete ..."
umgr.wait_units()
print "All CUs completed successfully!"
for cu in cu_set:
    print cu.stdout

### 3.4 Close Session

In [12]:
session.close()